#从对应的txt文件中读取word向量表和训练数据集
#embedding:words对应的150维向量(float)
#train_words:训练并列短语(string)
#train_types:训练并列短语每个词的词性(string)
#train_labels:训练并列短语对应的标记(int)

In [1]:
import pandas as pd
import numpy as np
import math
import theano
from sklearn import datasets
import pylab
import theano.tensor as T

In [40]:
#load embedding
embedding = pd.read_csv('embedding.csv')
embedding.index = embedding['Unnamed: 0'].tolist()
del embedding['Unnamed: 0']


#load training context
train_context_f = open('coordinate_data_150/train/context.txt')
train_context_input = train_context_f.readlines()
n = 0
train_words = [(["unknown"]*6) for i in range(int(len(train_context_input)/3))]
train_types = [(["unknown"]*6) for i in range(int(len(train_context_input)/3))]
words_index = 0
types_index = 0
while n < len(train_context_input):
    t = train_context_input[n].split()
    if n%3==0:
        for i in range(0,6):
            train_words[words_index][i] = t[i]
        words_index = words_index + 1
    elif n%3==1:
        for i in range(0,6):                                
            train_types[types_index][i] = t[i]
        types_index = types_index + 1
    n = n + 1
    
#load training labels
train_label_f = open('coordinate_data_150/train/labels.txt')
train_label_input = train_label_f.readlines()
n = 0
train_labels = [([-1]*3) for i in range(len(train_label_input))]
while n < len(train_label_input):
    t = train_label_input[n].split()
    for i in range(0,3):
        train_labels[n][i] = int(t[i])
    n = n + 1
    

In [6]:
#embedding.ix[train_types[0][0]]


In [7]:
#计算余弦距离的函数
def cosine_dis(vector1,vector2):
    '''
    @vector1 and @vector2 are two input vectors
    @Return the cosine distance between two vectors
    '''
    
    mm = 0
    for i in range(0,len(vector1)):
        mm = mm + vector1[i]*vector2[i]
    dd1 = 0
    dd2 = 0
    for i in range(0,len(vector1)):
        dd1 = dd1 + vector1[i]*vector1[i]
        dd2 = dd2 + vector2[i]*vector2[i]
    dd1 = math.sqrt(dd1)
    dd2 = math.sqrt(dd2)
    result = mm/(dd1*dd2)
    return result



In [8]:
#TODO: 根据词性来训练
#余弦距离的定义
#距离度量标准？？？

    
#第一步：单纯以context中对应位置的词性作为训练标准
#第二步：在词性的基础上加入词汇
#第三步：选取更多的度量方法


In [95]:
#神经网络模型

train_num = 10000; #训练量

#设置参数  
nn_input_dim=2 #输入神经元个数  
nn_output_dim=2 #输出神经元个数  
nn_hdim=100  #隐层节点数
epsilon=0.01 #learning rate  
reg_lambda=0.01 #正则化长度  

#参数初始化，设置为shared加速计算
w1=theano.shared(np.random.randn(nn_input_dim,nn_hdim),name="W1")  
b1=theano.shared(np.zeros(nn_hdim),name="b1")  
w2=theano.shared(np.random.randn(nn_hdim,nn_output_dim),name="W2")  
b2=theano.shared(np.zeros(nn_output_dim),name="b2")  

#前馈算法  
X=T.matrix('X')  #double类型的矩阵 
y=T.lvector('y') #int64类型的向量  
z1=X.dot(w1)+b1   #1 输入和w1的加权和
a1=T.tanh(z1)     #2 激活函数
z2=a1.dot(w2)+b2  #3 隐层输出和w2的加权和  
y_hat=T.nnet.softmax(z2) #4 激活函数  
loss_reg=1./train_num * reg_lambda/2 * (T.sum(T.square(w1))+T.sum(T.square(w2))) #5 正则化项    
loss=T.nnet.categorical_crossentropy(y_hat,y).mean()+loss_reg  #6 损失函数    
prediction=T.argmax(y_hat,axis=1) #7 预测结果  

forword_prop=theano.function([X],y_hat)  
calculate_loss=theano.function([X,y],loss)  
predict=theano.function([X],prediction)  

#求导  
dw2=T.grad(loss,w2)  
db2=T.grad(loss,b2)  
dw1=T.grad(loss,w1)  
db1=T.grad(loss,b1)  
#更新值  
gradient_step=theano.function(  
    [X,y],  
    updates=(  
        (w2,w2-epsilon*dw2),  
        (b2,b2-epsilon*db2),  
        (w1,w1-epsilon*dw1),  
        (b1,b1-epsilon*db1)  
  
    )  
)  

def build_model(num_passes=2000,print_loss=False):  
    w1.set_value(np.random.randn(nn_input_dim, nn_hdim) / np.sqrt(nn_input_dim))  
    b1.set_value(np.zeros(nn_hdim))  
    w2.set_value(np.random.randn(nn_hdim, nn_output_dim) / np.sqrt(nn_hdim))  
    b2.set_value(np.zeros(nn_output_dim))  
    for i in range(0,num_passes):  
        gradient_step(train_words_dis,train_Y)   #TODO change here
        if print_loss and i%10==0:  
            print("Loss after iteration %i: %f"%(i,calculate_loss(train_words_dis,train_Y)))  





In [76]:
#第一步：单纯以context中对应位置的词性作为训练标准
#计算context对应位置的余弦距离（2-4，3-5） TODO:改写这部分

train_words_dis = [([float]*nn_input_dim) for i in range(train_num)]


In [77]:
#embedding.ix[train_types[0][1]]

for i in range(0,train_num):
    train_words_dis[i][0] = cosine_dis(embedding.ix[train_types[i][1]],embedding.ix[train_types[i][3]])
    train_words_dis[i][1] = cosine_dis(embedding.ix[train_types[i][2]],embedding.ix[train_types[i][4]])
    if(i%100==0):
        print(i)
    



0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [85]:
train_Y = [([int]) for i in range(train_num)]
for i in range(train_num):
    train_Y[i] = train_labels[i][2]


In [96]:
build_model(print_loss=True)

Loss after iteration 0: 0.620067
Loss after iteration 10: 0.612536
Loss after iteration 20: 0.609048
Loss after iteration 30: 0.606344
Loss after iteration 40: 0.604018
Loss after iteration 50: 0.601949
Loss after iteration 60: 0.600075
Loss after iteration 70: 0.598360
Loss after iteration 80: 0.596778
Loss after iteration 90: 0.595315
Loss after iteration 100: 0.593956
Loss after iteration 110: 0.592692
Loss after iteration 120: 0.591513
Loss after iteration 130: 0.590413
Loss after iteration 140: 0.589385
Loss after iteration 150: 0.588423
Loss after iteration 160: 0.587522
Loss after iteration 170: 0.586677
Loss after iteration 180: 0.585884
Loss after iteration 190: 0.585138
Loss after iteration 200: 0.584436
Loss after iteration 210: 0.583776
Loss after iteration 220: 0.583153
Loss after iteration 230: 0.582565
Loss after iteration 240: 0.582010
Loss after iteration 250: 0.581485
Loss after iteration 260: 0.580988
Loss after iteration 270: 0.580517
Loss after iteration 280: 0.580

In [103]:
a = predict([[1,1],[0,1]])

In [104]:
a

array([1, 0])